## Get data

In [1]:
import time
import datetime
from yahoo_fin import stock_info as yf

def create_dataframe(asset, years_back, interval='1d'):
    return yf.get_data(
            ticker=asset+'=X', 
            start_date=(datetime.date.today() - datetime.timedelta(days=int(years_back*366))).strftime('%Y-%m-%d'), 
            end_date=time.strftime('%Y-%m-%d'), 
            interval=interval
        )


Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
df = create_dataframe(asset='USDKES', years_back=3)
df.tail()

,open,high,low,close,adjclose,volume,ticker
2024-08-13,128.000000,128.250000,127.750000,128.000000,128.000000,0.0,USDKES=X
2024-08-14,128.000000,129.229996,128.000000,128.000000,128.000000,0.0,USDKES=X
2024-08-15,129.699997,129.699997,127.750000,129.699997,129.699997,0.0,USDKES=X
2024-08-18,128.000000,128.500000,127.212067,128.000000,128.000000,0.0,USDKES=X
2024-08-20,128.479996,128.479996,128.229996,128.479996,128.479996,0.0,USDKES=X


## Preprocess the data

In [3]:
import numpy as np

def preprocess_data(df, days, scaler):
    df = df.copy()
    df['date'] = df.index
    df = df[['close', 'date']]
    df.loc[:, 'close'] = scaler.fit_transform(df[['close']])
    # Create the 'future' column by shifting 'close' by -days
    df['future'] = df['close'].shift(-days)
    df.dropna(inplace=True)
    return df

In [4]:
from collections import deque

def create_features_and_target(df, days, scaler):
    df = preprocess_data(df, days, scaler)
    # Capture the last 'days' number of 'close' values
    last_sequence = np.array(df[['close']].tail(days))
    sequence_data = []
    sequences = deque(maxlen=days)
    
    # Iterate through the DataFrame to create sequences of 'close' values and corresponding targets
    for entry, target in zip(df[['close', 'date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == days:
            sequence_data.append([np.array(sequences), target])
    
    # Prepare the last sequence by combining the most recent sequences with the last 'days' number of 'close' values
    last_sequence = list([s[:len(['close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)
    
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)

    features = np.array(X)
    # Ensure features only include the 'close' values
    features = features[:, :, :len(['close'])].astype(np.float32)
    target = np.array(y)

    return features, target, last_sequence


## Train a model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
days = 7

features, target, last_sequence = create_features_and_target(df, days, scaler)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=3)

In [6]:
import tensorflow

def train_model(features, target, epochs=100):
    model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.LSTM(60, return_sequences=True),
    tensorflow.keras.layers.Dropout(0.3),
    tensorflow.keras.layers.LSTM(120, return_sequences=False),
    tensorflow.keras.layers.Dropout(0.3),
    tensorflow.keras.layers.Dense(20),
    tensorflow.keras.layers.Dense(1),
    ])

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(features, target, epochs=epochs, verbose=False)
    return model

In [7]:
model = train_model(features, target)

## Predict prices

In [8]:
last_sequence = last_sequence[-days:]
predictions = model.predict(last_sequence)
predicted_price = scaler.inverse_transform(predictions)[0][0]
predicted_price

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step


129.34242